<a href="https://colab.research.google.com/github/Fadlanbima/ML_repo/blob/master/Search%20Engine/code/Search_Engine_model_using_cosine_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install and Import Module**

In [ ]:
!pip3 freeze > requirements.txt

In [1]:
!pip install Sastrawi
!pip install swifter
!pip install unidecode textblob sastrawi
!pip install spacy python-crfsuite unidecode textblob sastrawi

import numpy as np
import pandas as pd
import os
from  IPython import display

import pathlib
import shutil
import tempfile

!pip install -q git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import regularizers
from tensorflow.python.saved_model import saved_model

import pydot
import string
import re
import nltk
import Sastrawi
import pandas as pd
import swifter
import numpy as np
import tensorflow as tf

from nltk.tokenize import TweetTokenizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from collections import defaultdict
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import linear_kernel

nltk.download('popular')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 7.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 647 kB 9.4 MB/s 
     |████████████████████████████████| 281 kB 62.4 MB/s 
     |████████████████████████████████| 140 kB 69.5 MB/s 
  Created wheel for swifter: filename=swifter-1.1.3-py3-none-any.whl size=13078 sha256=ba7f4850f343e7aa434a1abb7feae9a1182cb3cd8c1f264c2416b5332069dafa
  Stored in directory: /root/.cache/pip/wheels/e8/1a/1b/46aa7b0fac16d3ce1bccaac14bfe4fad972642217ae78d8160
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 7.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 8.3 MB/s 
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloadi

True

# **Clone Data From Github**

In [3]:
!git clone https://github.com/Fadlanbima/ML_repo.git

Cloning into 'ML_repo'...
remote: Enumerating objects: 2870, done.
remote: Counting objects: 100% (271/271), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 2870 (delta 100), reused 263 (delta 92), pack-reused 2599
Receiving objects: 100% (2870/2870), 1.40 GiB | 38.43 MiB/s, done.
Resolving deltas: 100% (363/363), done.
Checking out files: 100% (2623/2623), done.


In [4]:
data = pd.read_csv('/content/ML_repo/data/Data Destinasi Wisatai.csv')

# **Some PreProcessing**

In [5]:
#casefolding
data['Detail_list'] = data['Detail'].str.lower()
data['jenis_wisata'] = data['Preferensi 2'].str.lower()

In [6]:
#memunculkan preferensi 2 sebanyak 5 kali untuk manambah bobot
data['jenis_wisata'] = data['jenis_wisata'].apply(lambda x: [x,x,x,x,x])

In [7]:
# mengubah list menjadi string
data['jenis_wisata'] = [' '.join(map(str, k)) for k in data['jenis_wisata']]

In [8]:
# menggabungkan data
data['data_ready'] = data['Detail_list']+ " " +data['jenis_wisata']+ " " +data['Destinasi']

In [10]:
#tokenizing
Tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)

def tokenization(text):
    teks= Tokenizer.tokenize(text)
    return teks

data['tokenized'] = data['data_ready'].apply(lambda x: tokenization(x))

In [11]:
factory = StopWordRemoverFactory()

Sastrawi_StopWords_id = factory.get_stop_words()

stopword = Sastrawi_StopWords_id + ['rt']

print(stopword)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [ ]:
#remove stopword
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
data['stopword_remove'] = data['tokenized'].apply(lambda x: remove_stopwords(x))

In [13]:
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
def wordLemmatizer(data):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    file_clean_k =pd.DataFrame()
    for index,entry in enumerate(data):
        
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if len(word)>1 and word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
            # The final processed set of words for each iteration will be stored in 'text_final'
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                #file_clean_k=file_clean_k.replace(to_replace ="\[.", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ="'", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace =" ", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ='\]', value = '', regex = True)
    return file_clean_k

In [14]:
df_clean = wordLemmatizer(data['stopword_remove']) 
df_clean

,Keyword_final
0,"['dago', 'dreampark', 'merupakan', 'salah', 's..."
1,"['gunung', 'tangkuban', 'perahu', 'merupakan',..."
2,"['museum', 'geologi', 'bandung', 'merupakan', ..."
3,"['museum', 'konferensi', 'asia', 'afrika', 'me..."
4,"['curug', 'citambur', 'sebuah', 'curug', 'air'..."
...,...
655,"['monumen', 'perang', 'dunia', 'ii', 'terletak..."
656,"['tugu', 'terletak', 'kampung', 'sota', 'sebua..."
657,"['taman', 'budaya', 'expo', 'waena', 'wisata',..."
658,"['taman', 'pholeuw', 'pholeuw', 'park', 'kabup..."


In [13]:
df_clean=df_clean.replace(to_replace ="\[.", value = '', regex = True)
df_clean=df_clean.replace(to_replace ="'", value = '', regex = True)
df_clean=df_clean.replace(to_replace =" ", value = '', regex = True)
df_clean=df_clean.replace(to_replace ='\]', value = '', regex = True)

In [14]:
data.insert(loc=3, column='Final_Keyword', value=df_clean['Keyword_final'].tolist())

In [15]:
data

,Destinasi,Detail,Jam Operasional,Final_Keyword,Biaya Masuk,Preferensi 2,Detail_list,jenis_wisata,data_ready,tokenized,stopword_remove
0,Dago Dreampark,Dago Dreampark merupakan salah satu tempat wis...,Senin - Jum'at ( 09.00 - 17.00 WIB ),"dago,dreampark,merupakan,salah,satu,tempat,wis...",Rp 27000 - Rp 32.000,wisata kota,dago dreampark merupakan salah satu tempat wis...,wisata kota wisata kota wisata kota wisata kot...,dago dreampark merupakan salah satu tempat wis...,"[dago, dreampark, merupakan, salah, satu, temp...","[dago, dreampark, merupakan, salah, satu, temp..."
1,Gunung Tangkuban Perahu,Gunung Tangkuban Perahu merupakan salah satu w...,Setiap Hari ( 07.00 - 17.00 WIB ),"gunung,tangkuban,perahu,merupakan,salah,satu,w...",Rp 22.000 - Rp 31.000,pegunungan dan pedesaan,gunung tangkuban perahu merupakan salah satu w...,pegunungan dan pedesaan pegunungan dan pedesaa...,gunung tangkuban perahu merupakan salah satu w...,"[gunung, tangkuban, perahu, merupakan, salah, ...","[gunung, tangkuban, perahu, merupakan, salah, ..."
2,Museum Geologi Bandung,Museum Geologi Bandung merupakan salah satu te...,Selasa - Kamis ( 10.00 - 13.00 WIB )\nSabtu - ...,"museum,geologi,bandung,merupakan,salah,satu,te...",Rp 2.000 - Rp 10.000,situs bersejarah dan religi,museum geologi bandung merupakan salah satu te...,situs bersejarah dan religi situs bersejarah d...,museum geologi bandung merupakan salah satu te...,"[museum, geologi, bandung, merupakan, salah, s...","[museum, geologi, bandung, merupakan, salah, s..."
3,Museum Konferensi Asia Afrika,Museum Konferensi Asia Afrika merupakan wisata...,"Selasa, Kamis, Sabtu, dan Minggu ( 09.00 - 15....","museum,konferensi,asia,afrika,merupakan,wisata...",Gratis,situs bersejarah dan religi,museum konferensi asia afrika merupakan wisata...,situs bersejarah dan religi situs bersejarah d...,museum konferensi asia afrika merupakan wisata...,"[museum, konferensi, asia, afrika, merupakan, ...","[museum, konferensi, asia, afrika, merupakan, ..."
4,Curug Citambur,Curug Citambur adalah sebuah curug atau air te...,Setiap Hari ( 09.00 - 22.00 WIB ),"curug,citambur,sebuah,curug,air,terjun,ketingg...",Rp 10.000 - Rp 15.000,pegunungan dan pedesaan,curug citambur adalah sebuah curug atau air te...,pegunungan dan pedesaan pegunungan dan pedesaa...,curug citambur adalah sebuah curug atau air te...,"[curug, citambur, adalah, sebuah, curug, atau,...","[curug, citambur, sebuah, curug, air, terjun, ..."
...,...,...,...,...,...,...,...,...,...,...,...
655,Monumen Perang Dunia II,Monumen Perang Dunia ke II terletak di Kampung...,Setiap Hari ( 09.00 - 17.00 WIT ),"monumen,perang,dunia,ii,terletak,kampung,paray...",Rp 25.000,wisata kota,monumen perang dunia ke ii terletak di kampung...,wisata kota wisata kota wisata kota wisata kot...,monumen perang dunia ke ii terletak di kampung...,"[monumen, perang, dunia, ke, ii, terletak, di,...","[monumen, perang, dunia, ii, terletak, kampung..."
656,Tugu Kembar Sabang-Merauke,"Tugu ini terletak di Kampung Sota, sebuah kamp...",Setiap Hari ( 24 Jam ),"tugu,terletak,kampung,sota,sebuah,kampung,perb...",Gratis,wisata kota,"tugu ini terletak di kampung sota, sebuah kamp...",wisata kota wisata kota wisata kota wisata kot...,"tugu ini terletak di kampung sota, sebuah kamp...","[tugu, ini, terletak, di, kampung, sota, ,, se...","[tugu, terletak, kampung, sota, ,, sebuah, kam..."
657,Taman Budaya EXPO Waena,Taman Budaya EXPO Waena,Setiap Hari ( 08.00 - 15.00 WIT ),"taman,budaya,expo,waena,wisata,kota,wisata,kot...",Gratis,wisata kota,taman budaya expo waena,wisata kota wisata kota wisata kota wisata kot...,taman budaya expo waena wisata kota wisata kot...,"[taman, budaya, expo, waena, wisata, kota, wis...","[taman, budaya, expo, waena, wisata, kota, wis..."
658,Taman Pholeuw,Taman Pholeuw atau Pholeuw Park Kabupaten Jaya...,Setiap Hari ( 09.00 - 20.00 WIT ),"taman,pholeuw,pholeuw,park,kabupaten,jayapura,...",Gratis,wisata kota,taman pholeuw atau pholeuw park kabupaten jaya...,wisata kota wisata ko

In [19]:
data.rename(columns={'Preferensi 2':'preferensi'}, inplace=True)

# **Split Data to Training, Testing and Validating**

In [20]:
train_data= data
#train_data.dropna(axis = 0, how ='any',inplace=True) 
train_data['Num_words_text'] = train_data['data_ready'].apply(lambda x:len(str(x).split())) 
mask = train_data['Num_words_text'] >2
train_data = train_data[mask]
print('===========Train Data =========')
print(train_data['preferensi'].value_counts())
print(len(train_data))
print('==============================')


test_data= data
#test_data.dropna(axis = 0, how ='any',inplace=True) 
test_data['Num_words_text'] = test_data['Detail'].apply(lambda x:len(str(x).split())) 
mask = test_data['Num_words_text'] >2
test_data = test_data[mask]
print('===========Test Data =========')
print(test_data['preferensi'].value_counts())
print(len(test_data))
print('==============================')

===========Train Data =========
wisata air                     219
pegunungan dan pedesaan        167
wisata kota                    137
situs bersejarah dan religi    137
Name: preferensi, dtype: int64
660
===========Test Data =========
wisata air                     219
pegunungan dan pedesaan        167
wisata kota                    137
situs bersejarah dan religi    137
Name: preferensi, dtype: int64
660


In [21]:

X_train, X_valid, y_train, y_valid = train_test_split(train_data['data_ready'].tolist(), train_data['preferensi'].tolist(), test_size=0.20,stratify = train_data['preferensi'].tolist(), random_state=0)


print('Train data len:'+str(len(X_train)))
print('Class distribution: '+str(Counter(y_train)))
print('Valid data len:'+str(len(X_valid)))
print('Class distribution: '+ str(Counter(y_valid)))




x_train=np.asarray(X_train)
x_valid = np.array(X_valid)
x_test =np.asarray(test_data['Detail'].tolist())

le = LabelEncoder()

train_labels = le.fit_transform(y_train)
train_labels = np.asarray( tf.keras.utils.to_categorical(train_labels))

valid_labels = le.transform(y_valid)
valid_labels = np.asarray( tf.keras.utils.to_categorical(valid_labels))

test_labels = le.transform(test_data['preferensi'].tolist())
test_labels = np.asarray(tf.keras.utils.to_categorical(test_labels))
list(le.classes_)


train_ds = tf.data.Dataset.from_tensor_slices((x_train,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid,valid_labels))
test_ds = tf.data.Dataset.from_tensor_slices((x_test,test_labels))

Train data len:528
Class distribution: Counter({'wisata air': 175, 'pegunungan dan pedesaan': 134, 'wisata kota': 110, 'situs bersejarah dan religi': 109})
Valid data len:132
Class distribution: Counter({'wisata air': 44, 'pegunungan dan pedesaan': 33, 'situs bersejarah dan religi': 28, 'wisata kota': 27})


In [22]:
print(y_train[:10])
train_labels = le.fit_transform(y_train)
print('Text to number')
print(train_labels[:10])
train_labels = np.asarray( tf.keras.utils.to_categorical(train_labels))
print('Number to category')
print(train_labels[:10])

['situs bersejarah dan religi', 'situs bersejarah dan religi', 'situs bersejarah dan religi', 'situs bersejarah dan religi', 'pegunungan dan pedesaan', 'pegunungan dan pedesaan', 'pegunungan dan pedesaan', 'wisata air', 'pegunungan dan pedesaan', 'situs bersejarah dan religi']
Text to number
[1 1 1 1 0 0 0 2 0 1]
Number to category
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]


In [23]:
count =0
print('======Train dataset ====')
for value,label in train_ds:
    count += 1
    print(value,label)
    if count==5:
        break
count =0
print('======Validation dataset ====')
for value,label in valid_ds:
    count += 1
    print(value,label)
    if count==5:
        break
print('======Test dataset ====')
for value,label in test_ds:
    count += 1
    print(value,label)
    if count==5:
        break  

======Train dataset ====
tf.Tensor(b'arboretum universitas riau memiliki luas \xc2\xb1 10 ha.arboretum terletak di kecamatan tampan kota pekanbaru provinsi riau dengan ketinggian 5-50 m dpl. arboretum ini berde- katan dengan tempat wisata yang minati oleh mahasiswa seperti jembatan kupu-kupu, air mancur, dan pendopo. situs bersejarah dan religi situs bersejarah dan religi situs bersejarah dan religi situs bersejarah dan religi situs bersejarah dan religi Arboretum Universitas Riau', shape=(), dtype=string) tf.Tensor([0. 1. 0. 0.], shape=(4,), dtype=float32)
tf.Tensor(b'museum siginjai jambi merupakan museum yang menyimpan berbagai koleksi benda-benda bersejarah, benda seni tradisi serta sejumlah instalasi edukasi pewarisan budaya lainya. situs bersejarah dan religi situs bersejarah dan religi situs bersejarah dan religi situs bersejarah dan religi situs bersejarah dan religi Museum SiGinjai', shape=(), dtype=string) tf.Tensor([0. 1. 0. 0.], shape=(4,), dtype=float32)
tf.Tensor(b'museum

# **Build Model semantic document search engine**

In [24]:
embedding = "https://tfhub.dev/google/nnlm-id-dim50-with-normalization/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [25]:
print(x_train[:1])
hub_layer(x_train[:1])

['arboretum universitas riau memiliki luas ± 10 ha.arboretum terletak di kecamatan tampan kota pekanbaru provinsi riau dengan ketinggian 5-50 m dpl. arboretum ini berde- katan dengan tempat wisata yang minati oleh mahasiswa seperti jembatan kupu-kupu, air mancur, dan pendopo. situs bersejarah dan religi situs bersejarah dan religi situs bersejarah dan religi situs bersejarah dan religi situs bersejarah dan religi Arboretum Universitas Riau']


<tf.Tensor: shape=(1, 50), dtype=float32, numpy=
array([[ 0.32198834, -0.10602163,  0.19620873, -0.7996683 , -0.21003628,
        -0.05695363, -0.3456228 , -0.23513353, -0.48728138,  0.29094782,
        -0.11146039,  0.5908577 ,  0.8881965 ,  0.679026  , -0.5132977 ,
         0.0475314 , -0.09545834,  0.23511581,  0.0194979 ,  0.21050121,
        -0.27257353, -0.22892703,  0.03140247, -0.5883649 ,  0.23408963,
         0.23111352, -0.00562019, -0.34477016,  0.62207973, -0.34004247,
        -0.3048622 , -0.4899819 ,  0.5665306 , -0.07839189, -0.03948281,
         0.61909664,  0.03375674, -0.16006008,  0.5627892 ,  0.3494001 ,
        -0.04040243,  0.48777497, -0.27528593,  0.520892  ,  0.17620948,
        -0.25595054, -0.14246643, -0.10206087,  0.32501423,  0.38059852]],
      dtype=float32)>

In [26]:
model = tf.keras.Sequential([hub_layer,
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dropout(0.1),
                             tf.keras.layers.Dense(8, activation='relu'),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(4,activation='sigmoid')
])


model.summary()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=["CategoricalAccuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                43930400  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_2 (Dense)             (None, 4)                 36        
                                                                 
Total params: 43,931,388
Trainable params: 43,931,388
No

In [27]:
epochs = 25

# Fit the model using the train and test datasets.
#history = model.fit(x_train, train_labels,validation_data= (x_test,test_labels),epochs=epochs )
history = model.fit(train_ds.shuffle(33).batch(10),
                    epochs= epochs ,
                    validation_data=valid_ds.batch(10),
                    verbose=1)

Epoch 1/25


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


53/53 [==============================] - 30s 528ms/step - loss: 1.2431 - categorical_accuracy: 0.4318 - val_loss: 1.0784 - val_categorical_accuracy: 0.6136
Epoch 2/25
53/53 [==============================] - 27s 510ms/step - loss: 0.9733 - categorical_accuracy: 0.6080 - val_loss: 0.7658 - val_categorical_accuracy: 0.8258
Epoch 3/25
53/53 [==============================] - 27s 511ms/step - loss: 0.6926 - categorical_accuracy: 0.7557 - val_loss: 0.4271 - val_categorical_accuracy: 0.9318
Epoch 4/25
53/53 [==============================] - 27s 517ms/step - loss: 0.4698 - categorical_accuracy: 0.8295 - val_loss: 0.2109 - val_categorical_accuracy: 0.9848
Epoch 5/25
53/53 [==============================] - 28s 524ms/step - loss: 0.2922 - categorical_accuracy: 0.9091 - val_loss: 0.1291 - val_categorical_accuracy: 0.9848
Epoch 6/25
53/53 [==============================] - 27s 515ms/step - loss: 0.2269 - categorical_accuracy: 0.9299 - val_loss: 0.0742 - val_categorical_accuracy: 1.0000
Epoch 7/2

# **Evaluating the Model**

In [28]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test,test_labels)
print("test loss, test acc:", results)


# Generate predictions (probabilities -- the output of the last layer)
# on test  data using `predict`
print("Generate predictions for all samples")
predictions = model.predict(x_test)
print(predictions)
predict_results = predictions.argmax(axis=1)

Evaluate on test data
21/21 [==============================] - 1s 69ms/step - loss: 0.6590 - categorical_accuracy: 0.7848
test loss, test acc: [0.6589697003364563, 0.7848485112190247]
Generate predictions for all samples
[[0.1832743  0.19118938 0.3879792  0.90753627]
 [0.93849206 0.10565957 0.36557752 0.4556166 ]
 [0.01271901 0.7553859  0.23746264 0.6846751 ]
 ...
 [0.19012153 0.28895319 0.22327876 0.9216564 ]
 [0.11091664 0.149407   0.21216276 0.9684036 ]
 [0.07508454 0.14287609 0.21856952 0.97617745]]


# **Saved Model**

In [29]:
MODEL = "saved_model"

In [30]:
model.save(MODEL)

INFO:tensorflow:Assets written to: rps_saved/assets


INFO:tensorflow:Assets written to: rps_saved/assets


# **Load the Model**

In [31]:
module_url = '/content/rps_saved'
#module_path ="/content/zettadevs/GoogleUSEModel/USE_4"
%time model = tf.saved_model.load(module_url)
#print ("module %s loaded" % module_url)

#Create function for using modeltraining
def embed(input):
    return model(input)

CPU times: user 1.53 s, sys: 842 ms, total: 2.37 s
Wall time: 2.29 s


In [32]:
Model_USE = embed(data.Destinasi[0:2500])

In [33]:
exported = tf.train.Checkpoint(v=tf.Variable(Model_USE))
exported.f = tf.function(
    lambda  x: exported.v * x,
    input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])
tf.saved_model.save(exported,'/content/modelgess/')

INFO:tensorflow:Assets written to: /content/modelgess/assets


INFO:tensorflow:Assets written to: /content/modelgess/assets


In [34]:
imported = tf.saved_model.load('/content/modelgess/')
loadedmodel = imported.v.numpy()

In [36]:
ls =[]
chunksize = 66
le =len(data.data_ready)
for i in range(0,le,chunksize):
    if(i+chunksize > le): 
        chunksize= le;
        ls.append(chunksize)
    else:
        a =i+chunksize
        ls.append(a)
ls
j=0
for i in ls:
    directory = "/content/modelgess/" + str(i)
    if not os.path.exists(directory):
        os.makedirs(directory)
    directory = "/content/modelgess/" + str(i)
    print(j,i) 
    m=embed(data.data_ready[j:i])
    exported_m = tf.train.Checkpoint(v=tf.Variable(m))
    exported_m.f = tf.function(
    lambda  x: exported_m.v * x,
    input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])

    tf.saved_model.save(exported_m,directory)
    j = i
    print(i)

0 66
INFO:tensorflow:Assets written to: /content/modelgess/66/assets


INFO:tensorflow:Assets written to: /content/modelgess/66/assets


66
66 132
INFO:tensorflow:Assets written to: /content/modelgess/132/assets


INFO:tensorflow:Assets written to: /content/modelgess/132/assets


132
132 198
INFO:tensorflow:Assets written to: /content/modelgess/198/assets


...

528
528 594
INFO:tensorflow:Assets written to: /content/modelgess/594/assets


INFO:tensorflow:Assets written to: /content/modelgess/594/assets


594
594 660
INFO:tensorflow:Assets written to: /content/modelgess/660/assets


INFO:tensorflow:Assets written to: /content/modelgess/660/assets


660


In [37]:
ar =[]
for i in ls:
    directory = "/content/modelgess/" + str(i)
    if os.path.exists(directory):
        print(directory)
        imported_m = tf.saved_model.load(directory)
        a= imported_m.v.numpy()
        #print(a)
        exec(f'load{i} = a')    

/content/modelgess/66
/content/modelgess/132
/content/modelgess/198
/content/modelgess/264
/content/modelgess/330
/content/modelgess/396
/content/modelgess/462
/content/modelgess/528
/content/modelgess/594
/content/modelgess/660


In [38]:
con_a = np.concatenate((load66,load132,load198,load264,load330,load396,load462,load528,load594,load660))
con_a.shape

(660, 4)

In [39]:
imported = tf.saved_model.load('/content/modelgess/')
loadedmodel =imported.v.numpy()
loadedmodel.shape

(660, 4)

# **Search engine for recomendation**

In [43]:
def recommendation(query):
    q = [query]
    # embed the query for calcluating the similarity
    Q_Train = embed(q)
    
    #imported_m = tf.saved_model.load('/content/rps_saved')
    #loadedmodel =imported_m.v.numpy()
    # Calculate the Similarity
    linear_similarities = linear_kernel(Q_Train, con_a).flatten() 
    #Sort top 10 index with similarity score
    Top_index_doc = linear_similarities.argsort()[:-50:-1]
    # sort by similarity score
    linear_similarities.sort()
    a = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        a.loc[i,'index'] = str(index)
        a.loc[i,'destinasi_rekomendasi'] = data['Destinasi'][index] ## Read File name with index from File_data DF
        a.loc[i,'biaya_masuk'] = data['Biaya Masuk'][index]
        a.loc[i,'jam_operasional'] = data['Jam Operasional'][index]
    for j,simScore in enumerate(linear_similarities[:-50:-1]):
        a.loc[j,'Score'] = simScore
    return a

In [46]:
recommendation('kota')

,index,destinasi_rekomendasi,biaya_masuk,jam_operasional,Score
0,454,Patung Nani Wartabone,Gratis,Setiap Hari ( 24 Jam ),1.085164
1,619,Kolam Pemandian Evu,Rp 20.000,Setiap Hari ( 06.00 - 18.00 WIT ),1.080657
2,522,Pasar Terapung Lok Baintan,Gratis,Setiap Hari ( 04.00 - 09.00 WITA ),1.078052
3,123,Taman Soekasada Ujung Water Palace,Rp 15.000,Setiap Hari ( 07.00 - 19.00 WITA ),1.066688
4,75,Taman Harmoni Keputih,Gratis,Setiap Hari ( 06.00 - 18.00 WIB ),1.063220
5,174,Rumah Budaya Sumba,Rp 15.000,Setiap Hari ( 24 Jam ),1.062356
6,204,Wisata Laiang Park Kota Solok,Rp 15.000 - Rp 75.000,Setiap Hari ( 08.00 - 18.00 WIB ),1.060532
7,521,Jembatan Gantung Nateh,Gratis,Setiap Hari ( 24 Jam ),1.057997
8,40,Nol Kilometer Yogyakarta,Gratis,Setiap Hari ( 24 Jam ),1.057987
9,338,Kebun Raya Liwa,Gratis,Selasa - Kamis ( 09.00 - 16.00 WIB )\nSabtu -...,1.057392


# **Download model to zip, for archive**

In [ ]:
!zip -r /content/model.zip /content/modelgess

  adding: content/modelgess/ (stored 0%)
  adding: content/modelgess/462/ (stored 0%)
  adding: content/modelgess/462/assets/ (stored 0%)
  adding: content/modelgess/462/saved_model.pb (deflated 71%)
  adding: content/modelgess/462/variables/ (stored 0%)
  adding: content/modelgess/462/variables/variables.index (deflated 30%)
  adding: content/modelgess/462/variables/variables.data-00000-of-00001 (deflated 10%)
  adding: content/modelgess/assets/ (stored 0%)
  adding: content/modelgess/660/ (stored 0%)
  adding: content/modelgess/660/assets/ (stored 0%)
  adding: content/modelgess/660/saved_model.pb (deflated 71%)
  adding: content/modelgess/660/variables/ (stored 0%)
  adding: content/modelgess/660/variables/variables.index (deflated 30%)
  adding: content/modelgess/660/variables/variables.data-00000-of-00001 (deflated 10%)
  adding: content/modelgess/330/ (stored 0%)
  adding: content/modelgess/330/assets/ (stored 0%)
  adding: content/modelgess/330/saved_model.pb (deflated 71%)
  ad

In [ ]:
from google.colab import files
files.download("/content/rps_saved.zip")

# **Convert Model to TFLite**

In [ ]:
# Convert model

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='/content/rps_saved', signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

In [ ]:
# Save the quantized model to file to the Downloads directory
f = open('docsearch_model.tflite', "wb")
f.write(tflite_model)
f.close()

# Download the digit classification model
from google.colab import files
files.download('docsearch_model.tflite')

print('`docsearch_model.tflite` has been downloaded')